# Assignment 3

### Imports

In [ ]:
import numpy as np
import scipy as sp
from scipy.optimize import dual_annealing, curve_fit
from scipy.integrate import solve_ivp, odeint
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from numba import jit, njit, prange
import math
import os
import random
import logging
from collections import Counter
import matplotlib as mpl

### Setup

In [ ]:
SEED = 33
np.random.seed(SEED)

os.makedirs('media', exist_ok=True)
logging.basicConfig(level=logging.ERROR)

if not os.path.isfile('predator-prey-data.csv'):
    raise FileNotFoundError("File 'predator-prey-data.csv' not found.")

DATA = pd.read_csv('predator-prey-data.csv').drop(columns=['Unnamed: 0'])
DATA_T, DATA_X, DATA_Y = DATA['t'].to_numpy(
), DATA['x'].to_numpy(), DATA['y'].to_numpy()

### Constants

In [ ]:
LOW_PARAM_BOUND = 0
UP_PARAM_BOUND = 2
MAX_STEPS_HILLCLIMING = 500
MAX_ITER_SA = 500
MSE_THRESH = 0.5

### Initial dataset

In [ ]:
plt.figure(figsize=(6, 4), layout='tight')
plt.scatter(DATA_T, DATA_X, s=10, label='Predator')
plt.scatter(DATA_T, DATA_Y, s=10, label='Prey')
plt.title('Predator-Prey dynamics of dataset')
plt.xlabel('Time $t$')
plt.ylabel('Population size $N$')
plt.legend(loc='upper right')
plt.grid(True)
plt.savefig('media/dataset.png', dpi=300)
plt.show()

### Predator-Prey ODEs

In [ ]:
def solve_lv(params, init_conditions, t):
    alpha, beta, gamma, delta = params

    def lv_odes(y, t):
        x, z = y
        dxdt = alpha*x - beta*x*z
        dzdt = delta*x*z - gamma*z
        return np.array([dxdt, dzdt])

    solution = odeint(lv_odes, init_conditions, t)
    return solution

### Optimization algorithms and utility

In [ ]:
def get_initial_random_guess(lower_bound=LOW_PARAM_BOUND, upper_bound=UP_PARAM_BOUND):
    alpha = random.uniform(lower_bound, upper_bound)
    beta = random.uniform(lower_bound, upper_bound)
    gamma = random.uniform(lower_bound, upper_bound)
    delta = random.uniform(lower_bound, upper_bound)
    return [alpha, beta, gamma, delta]


def get_neighbors(solution, step_size=0.1):
    """Calculate the neighbors of a solution, given a step size."""
    neighbors = []
    for i in range(len(solution)):
        # Add step_size to the parameter if it doesn't exceed PARAM_MAX
        if solution[i] + step_size <= UP_PARAM_BOUND:
            neighbors.append(
                [solution[j] if j != i else solution[i] + step_size for j in range(len(solution))])
        # Subtract step_size from the parameter if it doesn't become less than PARAM_MIN
        if solution[i] - step_size >= LOW_PARAM_BOUND:
            neighbors.append(
                [solution[j] if j != i else solution[i] - step_size for j in range(len(solution))])
    return neighbors


def choose_next_step(loss_function, current_step):
    neighbors = get_neighbors(current_step)
    return min(neighbors, key=loss_function)


def hill_climbing(loss_function, initial_step, max_steps):
    """Perform the Hill Climbing optimization algorithm."""
    current_step = initial_step
    history = [current_step]

    for _ in range(max_steps):
        try:
            next_step = choose_next_step(loss_function, current_step)
            if loss_function(next_step) >= loss_function(current_step):
                break
            history.append(next_step)
            current_step = next_step
        except Exception as e:
            logging.error(
                f"An error occurred during the hill climbing: {str(e)}")
            break

    return current_step, history


def classic_annealing(*args, **kwargs):
    return dual_annealing(*args, **kwargs, no_local_search=True)

### Objective functions

In [ ]:
def mse_objective(params):
    initial_conditions = [DATA_Y[0], DATA_X[0]]
    sim_data = solve_lv(params, initial_conditions, DATA_T)
    mse = mean_squared_error(DATA[['y', 'x']].values, sim_data)
    return mse


def mae_objective(params):
    initial_conditions = [DATA_Y[0], DATA_X[0]]
    sim_data = solve_lv(params, initial_conditions, DATA_T)
    mae = mean_absolute_error(DATA[['y', 'x']].values, np.abs(sim_data))
    return mae


def mse_objective_mod(params, data):
    valid_data = data.dropna(subset=['y', 'x'])
    if valid_data.empty:
        return float('inf')

    initial_conditions = [valid_data['y'].iloc[0], valid_data['x'].iloc[0]]
    sim_data = solve_lv(params, initial_conditions, valid_data['t'])
    return mean_squared_error(valid_data[['y', 'x']].values, sim_data)


def mae_objective_mod(params, data):
    valid_data = data.dropna(subset=['y', 'x'])
    if valid_data.empty:
        return float('inf')

    initial_conditions = [valid_data['y'].iloc[0], valid_data['x'].iloc[0]]
    sim_data = solve_lv(params, initial_conditions, valid_data['t'])
    return mean_absolute_error(valid_data[['y', 'x']].values, sim_data)


def smape_objective_mod(params, data):
    valid_data = data.dropna(subset=['y', 'x'])
    if valid_data.empty:
        return float('inf')

    initial_conditions = [valid_data['y'].iloc[0], valid_data['x'].iloc[0]]
    sim_data = solve_lv(params, initial_conditions, valid_data['t'])
    absolute_percentage_error = 2 * np.abs((sim_data - valid_data[['y', 'x']].values) /
                                           (np.abs(sim_data) + np.abs(valid_data[['y', 'x']].values)))
    return np.nanmean(absolute_percentage_error)

In [ ]:
bounds = [(LOW_PARAM_BOUND, UP_PARAM_BOUND)] * 4
initial_guess = get_initial_random_guess()
hc_optimizers = [hill_climbing]
hc_names = ['Hill Climbing']
sa_optimizers = [classic_annealing, dual_annealing]
sa_names = ['Classic Annealing', 'Dual Annealing']
optimizers = hc_optimizers + sa_optimizers
objectives = [mse_objective, mae_objective]
obj_names = ['MSE', 'MAE']
est_params = {opt.__name__: {obj.__name__: []
                             for obj in objectives} for opt in optimizers}

for obj in objectives:
    for opt in hc_optimizers:
        params, _ = hill_climbing(
            obj, initial_guess, max_steps=MAX_STEPS_HILLCLIMING)
        est_params[opt.__name__][obj.__name__] = params
        logging.info(
            f"Params of {opt.__name__} with {obj.__name__}: {[f'{p:.3f}' for p in params]} - Loss: {obj(params):.2e}")
    for opt in sa_optimizers:
        params = opt(obj, bounds=bounds, maxiter=MAX_ITER_SA).x
        est_params[opt.__name__][obj.__name__] = params
        logging.info(
            f"Params of {opt.__name__} with {obj.__name__}: {[f'{p:.3f}' for p in params]} - Loss: {obj(params):.2e}")

In [ ]:
alphas = [0.6, 0.8]


def plot_comparison_optimizers(hc_optimizers, sa_optimizers, objective=mse_objective, linestyles=['-', '--', ':'], colors=['tab:blue', 'tab:orange'], hc_names=hc_names, sa_names=sa_names, obj_names=obj_names):
    t_detail = np.linspace(0, 20, 1000)
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), layout='tight', sharey=True)
    fig.suptitle(
        f'Comparison of simulations with the experimental dataset by optimization algorithms using MSE')
    axs[0].scatter(DATA_T, DATA_X, s=10, color=colors[0])
    axs[0].scatter(DATA_T, DATA_Y, s=10, color=colors[1])
    for i, (opt, name) in enumerate(zip(hc_optimizers, hc_names)):
        params = est_params[opt.__name__][objective.__name__]
        initial_conditions = [DATA_Y[0], DATA_X[0]]
        sim_data = solve_lv(params, initial_conditions, t_detail)
        axs[0].plot(t_detail, sim_data[:, 1],
                    label=f'Predator ({name})', linestyle=linestyles[i], color=colors[0], linewidth=2, alpha=0.8)
        axs[0].plot(t_detail, sim_data[:, 0],
                    label=f'Prey ({name})', linestyle=linestyles[i], color=colors[1], linewidth=2, alpha=0.8)
    axs[0].set_title('Hill climbing-optimized model')
    axs[0].set_xlabel('Time $t$')
    axs[0].set_ylabel('Population size $N$')
    axs[0].legend(loc='upper right')
    axs[0].grid(True)
    axs[1].scatter(DATA_T, DATA_X, s=10, color=colors[0])
    axs[1].scatter(DATA_T, DATA_Y, s=10, color=colors[1])
    for i, (opt, name, alpha) in enumerate(zip(sa_optimizers, sa_names, alphas)):
        params = est_params[opt.__name__][objective.__name__]
        initial_conditions = [DATA_Y[0], DATA_X[0]]
        sim_data = solve_lv(params, initial_conditions, t_detail)
        axs[1].plot(t_detail, sim_data[:, 1],
                    label=f'Predator ({name})', linestyle=linestyles[i], color=colors[0], linewidth=1.5, alpha=alpha)
        axs[1].plot(t_detail, sim_data[:, 0],
                    label=f'Prey ({name})', linestyle=linestyles[i], color=colors[1], linewidth=1.5, alpha=alpha)
    axs[1].set_title('Simulated annealing-optimized model')
    axs[1].set_xlabel('Time $t$')
    axs[1].legend(loc='upper right')
    axs[1].grid(True)
    plt.savefig('media/comparison_optimizers.png', dpi=300)
    plt.show()


plot_comparison_optimizers(hc_optimizers, sa_optimizers)

Removing samples functionality

In [ ]:
def gen_sample_dataset(data, frac):
    data = data.copy()
    return data.sample(frac=frac).sort_values(by='t').reset_index(drop=True)


def batch_comparison_by_frac(fracs, n_samples=10, optimizer=classic_annealing, objective=mse_objective, initial_conditions=[DATA_Y[0], DATA_X[0]], data=DATA, **kwargs):
    results = []
    with tqdm(total=len(fracs)*n_samples) as pbar:
        for frac in fracs:
            mse = []
            for _ in range(n_samples):
                sample_data = gen_sample_dataset(data, frac)
                params = optimizer(objective, **kwargs).x
                sim_data = solve_lv(
                    params, initial_conditions, sample_data['t'])
                mse.append(mean_squared_error(
                    sample_data[['y', 'x']].values, sim_data))
                pbar.update()
            mean_mse, std_mse = np.mean(mse), np.std(mse)
            conf_int = sp.stats.norm.interval(
                0.95, loc=mean_mse, scale=std_mse)
            results.append([frac, mean_mse, std_mse, conf_int])
        logging.info(f"params: {params}")
        logging.info(
            f"Minimum and Maximum of sim_data: {sim_data.min()}, {sim_data.max()}")
        logging.info(
            f"MSE: {mean_squared_error(sample_data[['y', 'x']].values, sim_data)}")

    return pd.DataFrame(results, columns=['frac', 'mean', 'std', 'conf_int'])


def batch_comparison_by_frac_and_variable(data, fracs, data_variable, n_samples=10,
                                          optimizer=classic_annealing,
                                          objective=mse_objective_mod,
                                          **kwargs):
    results = []
    with tqdm(total=len(fracs) * n_samples) as pbar:
        for frac in fracs:
            mse = []
            for _ in range(n_samples):
                sample_data = gen_sample_dataset(data, frac)
                params = optimizer(lambda params: objective(
                    params, sample_data), **kwargs).x
                initial_conditions = [
                    sample_data['y'].iloc[0], sample_data['x'].iloc[0]]
                sim_data = solve_lv(
                    params, initial_conditions, sample_data['t'])
                mse.append(mean_squared_error(
                    sample_data[data_variable].values, sim_data[:, {'y': 0, 'x': 1}[data_variable]]))
                pbar.update()
            mean_mse, std_mse = np.mean(mse), np.std(mse)
            conf_int = sp.stats.norm.interval(
                0.95, loc=mean_mse, scale=std_mse)
            results.append([frac, mean_mse, std_mse, conf_int])
        logging.info(f"params: {params}")
        logging.info(
            f"Minimum and Maximum of sim_data: {sim_data.min()}, {sim_data.max()}")
        logging.info(
            f"MSE: {mean_squared_error(sample_data[data_variable].values, sim_data[:, {'y': 0, 'x': 1}[data_variable]])}")
    return pd.DataFrame(results, columns=['frac', 'mean', 'std', 'conf_int'])

In [ ]:
try:
    fracs = np.linspace(0.1, 1.0, 10)
    experiment_df = batch_comparison_by_frac(
        fracs=fracs, n_samples=20, bounds=bounds, maxiter=MAX_ITER_SA)
    print(experiment_df.head())
    plt.figure(figsize=(6, 4), layout='tight')
    plt.plot(experiment_df['frac'], experiment_df['mean'], label='Mean')
    plt.fill_between(experiment_df['frac'], experiment_df['conf_int'].apply(
        lambda x: x[0]), experiment_df['conf_int'].apply(lambda x: x[1]), alpha=0.2, label='95% confidence interval')
    plt.plot(experiment_df['frac'], [MSE_THRESH]*len(experiment_df['frac']),
             label='MSE threshold', linestyle='--', color='tab:red')
    plt.title('Mean and confidence interval of MSE by fraction of dataset')
    plt.xlabel('Fraction of dataset')
    plt.ylabel('MSE')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.savefig('media/frac_conf_mse_sa.png', dpi=300)
    plt.show()
except Exception as e:
    logging.error(f"An error occurred: {e}")

## Experiments and plots

In [ ]:
plt.figure(figsize=(6, 4), layout='tight')
for data_variable in ['y', 'x']:
    try:
        experiment_df = batch_comparison_by_frac_and_variable(fracs=fracs, data_variable=data_variable,
                                                              n_samples=20, bounds=bounds, maxiter=MAX_ITER_SA,
                                                              data=DATA, objective=mse_objective_mod)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
    plt.plot(experiment_df['frac'], experiment_df['mean'],
             label=f'{data_variable} (Mean $\\mu$)')
    plt.fill_between(experiment_df['frac'],
                     experiment_df['conf_int'].apply(lambda x: x[0]),
                     experiment_df['conf_int'].apply(lambda x: x[1]),
                     alpha=0.2, label=f'{data_variable} (95% CI)')
plt.title(f'Mean and confidence interval of MSE by fraction of dataset')
plt.xlabel('Fraction of dataset')
plt.ylabel('MSE')
plt.legend(loc='upper right')
plt.grid(True)
plt.savefig('media/frac_conf_mse_sa_by_var.png', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4), layout='tight')
for data_variable in ['y', 'x']:
    try:
        experiment_df = batch_comparison_by_frac_and_variable(fracs=fracs, data_variable='x',
                                                              n_samples=20, bounds=bounds, maxiter=MAX_ITER_SA,
                                                              data=DATA, objective=smape_objective_mod)
    except Exception as e:
        logging.error(f"An error occurred: {e}")

    plt.plot(experiment_df['frac'], experiment_df['mean'],
             label=f'{data_variable} (Mean $\\mu$)')
    plt.fill_between(experiment_df['frac'],
                     experiment_df['conf_int'].apply(lambda x: x[0]),
                     experiment_df['conf_int'].apply(lambda x: x[1]),
                     alpha=0.2, label=f'{data_variable} (95% CI)')
plt.title(f'Mean and confidence interval of sMAPE by fraction of dataset')
plt.xlabel('Fraction of dataset')
plt.ylabel('sMAPE')
plt.legend(loc='upper right')
plt.grid(True)
plt.savefig('media/frac_conf_smape_sa_by_var.png', dpi=300)
plt.show()

#### How many data-points from each time-series you could remove until you are not able to reverse-engineer the parameters any more?

First, try to find the importance of each time-series on reverse-engineering process by ﬁxing one time-series and removing data-points from the other one. In this way, you will ﬁnd the critical number of data-points for each time-series.

In [ ]:
def prune_data_series(data, variable, frac_to_remove):
    logging.info(
        f"\nAttempting to prune {frac_to_remove*100}% of '{variable}' data...")
    if not isinstance(frac_to_remove, (float, int)):
        raise TypeError('frac_to_remove should be a float or an integer.')
    if not 0 <= frac_to_remove <= 1:
        raise ValueError('frac_to_remove should be between 0 and 1 inclusive.')

    num_data_to_prune = int(len(data) * frac_to_remove)
    logging.info(f"Number of data points to prune: {num_data_to_prune}")

    pruned_data = data.copy()
    if num_data_to_prune > 0:
        prune_indices = np.random.choice(
            data.index, size=num_data_to_prune, replace=False)
        pruned_data.loc[prune_indices, variable] = np.nan
        logging.info(
            f"Pruned data. Remaining data points: {pruned_data.dropna(subset=[variable]).shape[0]}")
    else:
        logging.warning(
            "No data points to prune based on the fraction provided.")

    return pruned_data

In [ ]:
def optimize_with_pruned_data(pruned_data, optimizer, objective_function, bounds):
    logging.info("\nOptimizing with pruned data...")
    valid_data = pruned_data.dropna(subset=['y', 'x'])
    if valid_data.shape[0] < 2:
        logging.warning(
            "Not enough data points after pruning to perform optimization.")
        return None, float('inf')

    initial_conditions = [valid_data['y'].iloc[0], valid_data['x'].iloc[0]]
    try:
        result = optimizer(lambda params: objective_function(
            params, valid_data), bounds=bounds, maxiter=MAX_ITER_SA)
        logging.info("Optimization successful.")
        return result.x, result.fun
    except Exception as e:
        logging.error(f"An error occurred during optimization: {e}")
        return None, float('inf')

In [ ]:
# fractions = [round(float(i)/10, 1) for i in range(11)]
fractions = np.linspace(0.1, 1.0, 5)
objectives_mod = [mse_objective_mod, mae_objective_mod, smape_objective_mod]
# objectives_mod = [mse_objective_mod]
names = ['MSE', 'MAE', 'sMAPE']
n_sim = 15

plt.figure(figsize=(8, 5), layout='tight')
with tqdm(total=2*len(fractions)*len(objectives_mod)*n_sim) as pbar:
    for objective, name in zip(objectives_mod, names):
        results = []
        for variable in ['x', 'y']:
            for frac in fractions:
                losses = []
                for n in range(n_sim):
                    pruned_data = prune_data_series(DATA, variable, frac)
                    params, loss = optimize_with_pruned_data(
                        pruned_data, classic_annealing, objective_function=objective, bounds=bounds)
                    losses.append(loss)
                    pbar.update()
                results.append({
                    'variable': variable,
                    'fraction_removed': frac,
                    'loss': np.mean(losses)
                })
        results_df = pd.DataFrame(results)

        for variable in ['x', 'y']:
            variable_results = results_df[results_df['variable'] == variable]
            plt.plot(variable_results['fraction_removed'],
                     variable_results['loss'], label=f'{name} - {variable} data pruned')

    plt.xlabel('Fraction of data removed')
    plt.ylabel(f'Error after optimization')
    plt.legend()
    plt.title('Impact of Data Pruning on Parameter Estimation')
    plt.grid(True)
    plt.savefig(f'media/pruning_1.png', dpi=300)
    plt.show()

Then, if you combine two reduced time-series, could you still infer the parameters?

In [ ]:
fractions = np.arange(0.1, 1.0, 0.1)
n_sim = 15

with tqdm(total=2*len(fractions)*n_sim) as pbar:
    results = []
    for variable in ['x', 'y']:
        for frac in fractions:
            losses = []
            for n in range(n_sim):
                pruned_data = prune_data_series(DATA, variable, frac)
                params, loss = optimize_with_pruned_data(
                    pruned_data, classic_annealing, objective_function=mse_objective_mod, bounds=bounds)
                losses.append(loss)
                pbar.update()
            results.append({
                'variable': variable,
                'fraction_removed': frac,
                'loss': np.mean(losses)
            })
    results_df = pd.DataFrame(results)

    for variable in ['x', 'y']:
        variable_results = results_df[results_df['variable'] == variable]
        plt.plot(variable_results['fraction_removed'],
                 variable_results['loss'], label=f'{variable} data pruned')

    plt.xlabel('Fraction of data removed')
    plt.ylabel(f'MSE after optimization')
    plt.legend()
    plt.title('Impact of Data Pruning on Parameter Estimation')
    plt.grid(True)
    plt.show()

In [ ]:
def prune_both_series(data, frac_to_remove):
    pruned_data = data.copy()
    prune_indices = np.random.choice(data.index, size=int(
        len(data) * frac_to_remove), replace=False)
    pruned_data.loc[prune_indices, ['x', 'y']] = np.nan
    return pruned_data


def run_optimization_on_pruned_data(fracs, n_samples=10, objectives=[mse_objective_mod, mae_objective_mod, smape_objective_mod]):
    results = []
    for frac in fracs:
        for objective in objectives:
            for _ in range(n_samples):
                pruned_data = prune_both_series(DATA, frac)
                initial_guess = get_initial_random_guess()

                if objective in [mse_objective_mod, mae_objective_mod, smape_objective_mod]:
                    # For modified objectives, pass both parameters and data
                    params, _ = hill_climbing(lambda params: objective(
                        params, pruned_data), initial_guess, max_steps=MAX_STEPS_HILLCLIMING)
                else:
                    # For standard objectives, pass only parameters
                    params, _ = hill_climbing(
                        objective, initial_guess, max_steps=MAX_STEPS_HILLCLIMING)

                loss = objective(params, pruned_data)
                results.append(
                    {'frac': frac, 'objective': objective.__name__, 'loss': loss})
    return pd.DataFrame(results)


fractions = np.arange(0.1, 1.0, 0.1)
experiment_results = run_optimization_on_pruned_data(fractions, n_samples=15)

for objective_name in ['mse_objective_mod', 'mae_objective_mod', 'smape_objective_mod']:
    subset = experiment_results[experiment_results['objective']
                                == objective_name]
    plt.plot(subset['frac'], subset['loss'], label=objective_name)

plt.xlabel('Fraction of data removed')
plt.ylabel('Loss after optimization')
plt.legend()
plt.title('Parameter Estimation with Combined Data Reduction')
plt.grid(True)
plt.show()

Does removing every data-point have the same effect on reverse-engineering process? In other words, what are the points that you could remove safely without affecting the inferance procedure? and what are those with critical effects on the process?

In [ ]:
def leave_one_out_analysis(data, initial_guess, bounds, maxiter):
    importance_scores = []

    for i in tqdm(range(len(data)), desc="Analyzing Data Points"):
        # Exclude one data point
        reduced_data = data.drop(index=i)
        t_reduced, x_reduced, y_reduced = reduced_data['t'].to_numpy(
        ), reduced_data['x'].to_numpy(), reduced_data['y'].to_numpy()

        def mse_objective_reduced(params):
            initial_conditions = [y_reduced[0], x_reduced[0]]
            simulated = solve_lv(params, initial_conditions, t_reduced)
            mse = np.mean((simulated - reduced_data[['y', 'x']].values) ** 2)
            return mse

        # Fit the model on the reduced dataset
        result = classic_annealing(
            mse_objective_reduced, bounds=bounds, maxiter=maxiter)
        params = result.x
        mse = mse_objective_reduced(params)
        importance_scores.append((i, mse))

    return importance_scores


mse_list = []
n_sims = 10
for sims in range(n_sims):
    importance_scores = leave_one_out_analysis(
        DATA, initial_guess, bounds, MAX_ITER_SA)
    importance_scores.sort(key=lambda x: x[1], reverse=True)
    mse_list.append(importance_scores[:10])

In [ ]:
# Flatten the list of MSEs
flat_mse_list = [item for sublist in mse_list for item in sublist]

# Count the frequency of each data point
data_point_frequencies = Counter(
    [datapoint for datapoint, mse in flat_mse_list])
total_entries = len(flat_mse_list)
probability_distribution = {
    datapoint: freq / total_entries for datapoint, freq in data_point_frequencies.items()}

# Create Dataframe
probability_df = pd.DataFrame(list(probability_distribution.items()), columns=[
                              'Data Point', 'Probability'])

probability_df.head()

In [ ]:
mpl.rcParams.update({'font.size': 13})

# Sort the DataFrame for better visualization
probability_df_sorted = probability_df.sort_values(
    by='Probability', ascending=False)

# Plotting
plt.figure(figsize=(15, 6))  # You can adjust the figure size as needed
plt.bar(probability_df_sorted['Data Point'],
        probability_df_sorted['Probability'], color='blue')

plt.xlabel('Data Point')
plt.ylabel('Probability')
plt.title('Probability Distribution of Data Points Being In the Top 10 Highest Influence from 10 Simulations')
plt.savefig('media/probability_distribution_datap.png')

plt.show()

In [ ]:
# All points that occur in the probability DF will be highlighted
n_highlight = len(probability_df)

# Sort the DataFrame
high_probability_indices = probability_df.sort_values(
    by='Probability', ascending=False).head(n_highlight)['Data Point']

# Plot the original data
plt.scatter(DATA['t'], DATA['x'], label='Predator', alpha=0.7)
plt.scatter(DATA['t'], DATA['y'], label='Prey', alpha=0.7)

# Highlight the points
for idx in high_probability_indices:
    plt.scatter(DATA.loc[idx, 't'], DATA.loc[idx, 'x'],
                color='blue', edgecolor='black', s=50)
    plt.scatter(DATA.loc[idx, 't'], DATA.loc[idx, 'y'],
                color='red', edgecolor='black', s=50)

plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Predator-Prey Data with High Impact Points Highlighted')
plt.legend(loc="upper right", fontsize='small')
plt.savefig('media/high_probability_points_plot.png', dpi=300)
plt.show()

In [ ]:
n_highlight = 10

# Plot the original data
plt.scatter(DATA['t'], DATA['x'], label='Predator', alpha=0.7)
plt.scatter(DATA['t'], DATA['y'], label='Prey', alpha=0.7)

# Highlight the points with highest probability
for idx in high_probability_indices:
    plt.scatter(DATA.loc[idx, 't'], DATA.loc[idx, 'x'],
                color='blue', edgecolor='black', s=50)
    plt.scatter(DATA.loc[idx, 't'], DATA.loc[idx, 'y'],
                color='red', edgecolor='black', s=50)

plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Predator-Prey Data with the Highest Probability Points Highlighted')
plt.legend(loc="upper right", fontsize='small')
plt.savefig('media/high_probability_points_plot.png', dpi=300)
plt.show()

#### \<Extra question\>